In [1]:
import lxml.html
import time
import re
from tqdm import tqdm
from stanza.server import CoreNLPClient
import pickle
from projectmir.xmldocument import XMLDocument
from projectmir.functions import find_identifier_definition

from pathlib import Path
import copy

In [2]:
p_path = './data/test_latexml/'
p = Path(p_path)
x= list(p.glob('*.html'))#[2:3]#[0:1]
document_IDs = [x_.name[:-5] for x_ in x]

doc = [[]]*len(document_IDs)
for i, document_ID in tqdm(enumerate(document_IDs)):
    document_path = p_path + document_ID + '.html'
    doc_ = find_identifier_definition(document_path)
    doc[i] = doc_

0it [00:00, ?it/s]

elapsed time : 0.0000501 seconds ---
process document
elapsed time : 0.0028751 seconds ---
Number of math components is 50
elapsed time : 0.0317910 seconds ---
elapsed time : 0.0678861 seconds ---
Starting server with command: java -Xmx16G -cp /Users/kato/GoogleDrive/project-mir/data/resources/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 600000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-a23257077c74415b.props -preload tokenize,ssplit,pos
elapsed time : 1.8867600 seconds ---
Starting server with command: java -Xmx16G -cp /Users/kato/GoogleDrive/project-mir/data/resources/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 600000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-435d1309cc2142d7.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref


1it [00:33, 33.45s/it]

extract_noun_phrases
elapsed time : 33.4456689 seconds ---
elapsed time : 33.4461119 seconds ---


In [3]:
for i, x in enumerate(doc[0].candidate_noun_phrases_list):
    # print(x)
    print(len(x))

2
1
1
2
1
1
1
3
2
4
0
0
2
0
1


In [52]:
i=8
print(doc[0].identifiers[i])
doc[0].description_candidate[i]

U_int


[]

In [51]:
i=8
print(doc[0].identifiers[i])
doc[0].sentences[i]

U_int


['For a pure liquid at low or moderate pressures, the internal energy is approximately equal to the enthalpy, MATH0008≈MATH0009, and MATH0009 depends only on temperature (Sandler, 2006).',
 'Consequently, in the subsequent development, we assume that MATH0008=MATH0009 and U^_int=MATH0009^ where the caret (^) means per unit mass.']

In [19]:
for i, identifier in enumerate(doc[0].identifiers):
    print(identifier)
    ii=0
    for x in doc[0].candidate_noun_phrases_list[i]:
        for x_ in x:
            print(ii, x_.replace('\n', ''))
            ii+=1



w_i
0 (NP (DT The) (JJ liquid) (NN inlet) (NN stream))
1 (NP  (NP (DT a) (JJ single) (NN component))  (PP (IN with)    (NP      (NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))      (CC and)      (NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001)))))
2 (NP (DT a) (JJ single) (NN component))
3 (NP  (NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))  (CC and)  (NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001)))
4 (NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))
5 (NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001))
6 (NP (DT The)  (NP    (NP (NN inlet))    (CC and)    (NP (NN outlet) (NN flow) (NNS rates))))
7 (NP  (NP (NN inlet))  (CC and)  (NP (NN outlet) (NN flow) (NNS rates)))
8 (NP (NN inlet))
9 (NP (NN outlet) (NN flow) (NNS rates))
10 (NP  (NP (NN MATH0000) (JJ =) (NN MATH0004))  (CC and)  (NP (DT the) (JJ liquid) (NN holdup) (NN MATH0005)))
11 (NP (NN MATH0000) (JJ =) (NN MATH0004))
12 (NP (DT the) (JJ liquid) (NN holdup) (NN MATH0005))
T_i
0 (NP (DT T

In [57]:
import math
sigma_d = math.sqrt(12/math.log(2))
sigma_s = 2 / math.sqrt(math.log(2))
a = 1
b = 1
c = 0.1
d = 1
print(f'{sigma_d=}')
print(f'{sigma_s=}')

sigma_d=4.1608100762552915
sigma_s=2.4022448175728996


In [58]:
def rank_based_pagel(D, n, tf):
    R_sigma_d = math.exp(- (D**2 - 1) / 2 / sigma_d**2)
    R_sigma_s = math.exp(- (n**2 - 1) / 2 / sigma_s**2)
    print(f'{R_sigma_d=}, {R_sigma_s=}')
    return  (a * R_sigma_d + b * R_sigma_s + c * tf) / (a + b + c)

In [60]:
def rank_proposed(D, n, tf, M=0):
    R_sigma_d = math.exp(- (D**2 - 1) / 2 / sigma_d**2)
    R_sigma_s = math.exp(- (n**2 - 1) / 2 / sigma_s**2)
    print(f'{R_sigma_d=}, {R_sigma_s=}')
    return  (a * R_sigma_d + b * R_sigma_s + c * tf + d * M) / (a + b + c + d)

In [61]:
# N_f
print('rank proposed by Pagel')
print(rank_based_pagel(1, 0, 1))
print(rank_based_pagel(3, 0, 1))
print(rank_based_pagel(1, 0, 1))

print('rank proposed in this research')
print(rank_proposed(1, 0, 1, 0))
print(rank_proposed(3, 0, 1, 0))
print(rank_proposed(1, 0, 1, 0))

rank proposed by Pagel
R_sigma_d=1.0, R_sigma_s=1.0905077326652577
1.0430989203167895
R_sigma_d=0.7937005259840998, R_sigma_s=1.0905077326652577
0.9448610755473131
R_sigma_d=1.0, R_sigma_s=1.0905077326652577
1.0430989203167895
rank proposed in this research
R_sigma_d=1.0, R_sigma_s=1.0905077326652577
0.7066153976339542
R_sigma_d=0.7937005259840998, R_sigma_s=1.0905077326652577
0.6400671802094702
R_sigma_d=1.0, R_sigma_s=1.0905077326652577
0.7066153976339542


#### U_int
- a pure liquid at low or moderate pressure  
- a pure liquid  
- low or moderate pressures  
- the internal energy  
- the enthalpy  
- temperature  
- the subsequent development  
- we  
- the caret (^)  
- unit mass

In [62]:
# U_int
print('rank proposed by Pagel')
print(rank_based_pagel(9, 0, 1))
print(rank_based_pagel(14, 0, 1))
print(rank_based_pagel(9, 0, 1))
print(rank_based_pagel(6, 0, 1))
print(rank_based_pagel(0, 0, 1))
print(rank_based_pagel(7, 0, 1))
print(rank_based_pagel(3, 1, 1))
print(rank_based_pagel(2, 1, 1))
print(rank_based_pagel(7, 1, 1))
print(rank_based_pagel(14, 1, 1))

print('rank proposed in this research')
print(rank_proposed(9, 0, 1))
print(rank_proposed(14, 0, 1))
print(rank_proposed(9, 0, 1))
print(rank_proposed(6, 0, 1, 1))
print(rank_proposed(0, 0, 1))
print(rank_proposed(7, 0, 1))
print(rank_proposed(3, 1, 1))
print(rank_proposed(2, 1, 1))
print(rank_proposed(7, 1, 1))
print(rank_proposed(14, 1, 1, 1))

rank proposed by Pagel
R_sigma_d=0.09921256574801256, R_sigma_s=1.0905077326652577
0.6141525230539382
R_sigma_d=0.003582047043768253, R_sigma_s=1.0905077326652577
0.5686141808138219
R_sigma_d=0.09921256574801256, R_sigma_s=1.0905077326652577
0.6141525230539382
R_sigma_d=0.363913295710547, R_sigma_s=1.0905077326652577
0.7402004897027642
R_sigma_d=1.029302236643492, R_sigma_s=1.0905077326652577
1.0570523663375
R_sigma_d=0.2500000000000001, R_sigma_s=1.0905077326652577
0.6859560631739323
R_sigma_d=0.7937005259840998, R_sigma_s=1.0
0.9017621552305236
R_sigma_d=0.9170040432046712, R_sigma_s=1.0
0.9604781158117482
R_sigma_d=0.2500000000000001, R_sigma_s=1.0
0.6428571428571429
R_sigma_d=0.003582047043768253, R_sigma_s=1.0
0.5255152604970325
rank proposed in this research
R_sigma_d=0.09921256574801256, R_sigma_s=1.0905077326652577
0.4160388059397646
R_sigma_d=0.003582047043768253, R_sigma_s=1.0905077326652577
0.3851902515190406
R_sigma_d=0.09921256574801256, R_sigma_s=1.0905077326652577
0.4160

In [ ]:
doc_original = copy.deepcopy(doc[0])

In [ ]:
x = copy.deepcopy(doc_original)

In [ ]:
print(id(doc[0]))
print(id(doc_original))
print(id(x))

In [ ]:
tree = lxml.html.parse(x.path)
html = tree.getroot()
print('Number of math components is {}'.format(
    len(html.cssselect('math'))))
identifiers = []

def is_identifier(math_component):
    is_mi = (math_component.tag == 'mi')
    is_math_component_len_1 = (len(math_component.text_content()) == 1)
    is_italic = (math_component.get('mathvariant') == 'italic')
    return is_mi and (is_math_component_len_1 or is_italic)

for html_math in html.cssselect('math'):
    # variable with subscript and superscript
    for html_math_msubsup in html_math.cssselect('msubsup'):
        html_math_msubsup_component = [
            x for x in html_math_msubsup.iterchildren()]
        math_txt = [x.text_content()
                    for x in html_math_msubsup_component]
        math_txt = math_txt[0] + '_' + math_txt[1] + '^' + math_txt[2]
        if is_identifier(html_math_msubsup) and (
                math_txt not in identifiers):
            identifiers.append(math_txt)


    # variable with subscript
    for html_math_msub in html_math.cssselect('msub'):
        html_math_msub_component = [
            x for x in html_math_msub.iterchildren()]
        math_txt = '_'.join([x.text_content()
                                for x in html_math_msub_component])
        # if is_identifier(
        #         html_math_msub_component[0]) and (
        #         math_txt not in identifiers):
        #     identifiers.append(math_txt)
        if is_identifier(
                html_math_msub_component[
                    [x.tag for x in html_math_msub_component].index('mi')]) and (
                math_txt not in identifiers):
            identifiers.append(math_txt)
        print(html_math_msub)
        html_math_msub.drop_tree()
        print(html_math_msub)
        print(html_math_msub_component)

        reg_string = lxml.html.tostring(
            html_math_msub, encoding='unicode')
        print(reg_string)
        print(is_identifier(html_math_msub_component[0]))

    # variable with superscript
    for html_math_msup in html_math.cssselect('msup'):
        html_math_msup_component = [
            x for x in html_math_msup.iterchildren()]
        math_txt = '^'.join([x.text_content()
                                for x in html_math_msup_component])
        if is_identifier(
                html_math_msup_component[0]) and (
                math_txt not in identifiers):
            identifiers.append(math_txt)

    # variable without subscript and superscript
    for html_math_mi in html_math.cssselect('mi'):
        math_txt = html_math_mi.text_content()
        if is_identifier(html_math_mi) and (
                math_txt not in identifiers):
            identifiers.append(math_txt)

x.identifiers = identifiers
x.text = lxml.html.fromstring(x.body).text_content()


In [ ]:
tree = lxml.html.parse(x.path)
html = tree.getroot()
print('Number of math components is {}'.format(
    len(html.cssselect('math'))))
reg_string_list = []
identifiers = []

def is_identifier(math_component):
    is_mi = (math_component.tag == 'mi')
    is_math_component_len_1 = (len(math_component.text_content()) == 1)
    is_italic = (math_component.get('mathvariant') == 'italic')
    return is_mi and (is_math_component_len_1 or is_italic)


# TODO: variable with subscript is not changed,
# because before change, the variable without subscript is changed.
for html_math in html.cssselect('math'):
    # variable with subscript and superscript
    for html_math_msubsup in html_math.cssselect('msubsup'):
        html_math_msubsup_component = [
            x for x in html_math_msubsup.iterchildren()]
        math_txt = [x.text_content()
                    for x in html_math_msubsup_component]
        math_txt = math_txt[0] + '_' + math_txt[1] + '^' + math_txt[2]
        if is_identifier(html_math_msubsup) and (
                math_txt not in identifiers):
            identifiers.append(math_txt)
        html_math_msubsup.drop_tree()
        reg_string = lxml.html.tostring(
            html_math_msubsup, encoding='unicode')
        if is_identifier(html_math_msubsup):
            reg_string_list.append(
                (math_txt, reg_string, 'MATH{:04d}'.format(
                    identifiers.index(math_txt))))

        else:
            reg_string_list.append((math_txt, reg_string, math_txt))

    # variable with subscript
    for html_math_msub in html_math.cssselect('msub'):
        html_math_msub_component = [
            x for x in html_math_msub.iterchildren()]
        math_txt = '_'.join([x.text_content()
                             for x in html_math_msub_component])
        if is_identifier(
                html_math_msub_component[0]) and (
                math_txt not in identifiers):
            identifiers.append(math_txt)
        html_math_msub.drop_tree()
        reg_string = lxml.html.tostring(
            html_math_msub, encoding='unicode')
        if is_identifier(html_math_msub_component[0]):
            reg_string_list.append(
                (math_txt, reg_string, 'MATH{:04d}'.format(
                    identifiers.index(math_txt))))

        else:
            reg_string_list.append(
                (math_txt, reg_string, math_txt))

    # variable with superscript
    for html_math_msup in html_math.cssselect('msup'):
        html_math_msup_component = [
            x for x in html_math_msup.iterchildren()]
        math_txt = '^'.join([x.text_content()
                             for x in html_math_msup_component])
        if is_identifier(
                html_math_msup_component[0]) and (
                math_txt not in identifiers):
            identifiers.append(math_txt)
        html_math_msup.drop_tree()
        reg_string = lxml.html.tostring(
            html_math_msup, encoding='unicode')
        if is_identifier(html_math_msup_component[0]):
            reg_string_list.append(
                (math_txt, reg_string, 'MATH{:04d}'.format(
                    identifiers.index(math_txt))))

        else:
            reg_string_list.append(
                (math_txt,
                 reg_string,
                 math_txt)
            )

    # variable without subscript and superscript
    for html_math_mi in html_math.cssselect('mi'):
        math_txt = html_math_mi.text_content()
        if is_identifier(html_math_mi) and (
                math_txt not in identifiers):
            identifiers.append(math_txt)
        html_math_mi.drop_tree()
        reg_string = lxml.html.tostring(
            html_math_mi, encoding='unicode')
        if is_identifier(html_math_mi):
            reg_string_list.append(
                (math_txt,
                 reg_string,
                 'MATH{:04d}'.format(identifiers.index(math_txt))
                 )
            )

        else:
            reg_string_list.append(
                (math_txt,
                 reg_string,
                 math_txt)
            )
reg_string_list = list(set(reg_string_list))
reg_string_list = sorted(reg_string_list, key=lambda x:x[0], reverse=True)
print(reg_string_list)
for reg_string in reg_string_list:
    x.body.replace(reg_string[1], reg_string[2])
x.identifiers = identifiers
x.text = lxml.html.fromstring(x.body).text_content()
reg_string_list.sort(key=lambda x: x[0], reverse=True)

In [ ]:
with open('extracted_text.txt', 'w') as f:
    f.write(x.text)

In [ ]:
doc_original.text